<a href="https://colab.research.google.com/github/MuriloWollmann/Projeto-Final-de-Fabrica-de-Software---Automatizacao-da-Busca-e-Tratamento-de-Dados-de-Saude-Publica/blob/main/PFS_Dados_de_Sa%C3%BAde_P%C3%BAblica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Baixa algumas tabelas que são de interesse, concatena as 4 primeiras (por local de atendimento) e as 4 últimas (por local de residência).
# Para baixá-las, é só rodar o código e abrir o diretório (ícone de pasta na esquerda) - Vão ser as duas primeiras.
# Se quiser abri-las no próprio colab, crie duas células de código e rode as respectivas variáveis: concatenadaAtendimento e concatenadaResidencia.

# Instala o pacote google-colab-selenium para automação web no Google Colab
%pip install -q google-colab-selenium

# Importações necessárias
import google_colab_selenium as gs # Selenium para Google Colab
import os # Manipulação do sistema de arquivos (diretório)
import pandas as pd # Manipulação de dataframes
import time # Manipulação de tempo (para pausas)
from selenium import webdriver # Webdriver para automação de navegador
from selenium.webdriver.common.by import By # Localização de elementos
from selenium.webdriver.common.action_chains import ActionChains # Cadeias de ações no Selenium
from selenium.webdriver.common.keys import Keys # Simulação de teclas
from selenium.webdriver.support.ui import WebDriverWait # Esperas explícitas
from selenium.webdriver.support import expected_conditions as EC # Condições esperadas para esperas explícitas

# Classe para realizar a automação web
class WebScraper:
    def __init__(self):
        self.driver = gs.Chrome() # Inicia o navegador Chrome no Google Colab
        self.wait = WebDriverWait(self.driver, 10) # Configura espera explícita de 10 segundos
        self.driver.get('https://datasus.saude.gov.br/informacoes-de-saude-tabnet/') # Abre a URL do Datasus

    def get_elemento(self, xpath):
        return self.wait.until(EC.presence_of_element_located((By.XPATH, xpath))) # Espera até o elemento estar presente

    def clicar_elemento(self, xpath):
        elemento = self.get_elemento(xpath) # Localiza o elemento
        elemento.click() # Clica no elemento
        return elemento

    def escrever_no_campo(self, xpath, texto):
        campo = self.get_elemento(xpath) # Localiza o campo
        campo.send_keys(texto) # Escreve no campo
        return campo

    def limpar_campo(self, xpath):
        campo = self.get_elemento(xpath) # Localiza o campo
        campo.clear() # Limpa o campo

    def aguardar_visibilidade(self, xpath):
        return self.wait.until(EC.visibility_of_element_located((By.XPATH, xpath))) # Espera até o elemento estar visível

    def tirar_screenshot(self, nome_arquivo):
        self.driver.save_screenshot(nome_arquivo) # Salva uma captura de tela

    def selecionar_multiplas_opcoes(self, *xpaths):
        action = ActionChains(self.driver) # Inicia uma cadeia de ações
        action.key_down(Keys.CONTROL) # Pressiona a tecla CTRL
        for xpath in xpaths:
            self.aguardar_visibilidade(xpath) # Espera a visibilidade do elemento
            elemento = self.get_elemento(xpath) # Localiza o elemento
            action.click(elemento) # Clica no elemento
        action.key_up(Keys.CONTROL) # Solta a tecla CTRL
        action.perform() # Executa a cadeia de ações

    def trocar_para_nova_janela(self, janela_indice):
        nova_janela = self.driver.window_handles[janela_indice] # Obtém o identificador da nova janela
        self.driver.switch_to.window(nova_janela) # Troca para a nova janela

    def fechar_navegador(self):
        self.driver.quit() # Fecha o navegador

    def aguardar(self, segundos):
        time.sleep(segundos) # Aguarda um determinado número de segundos


def atendimento():
    try:
        # Inicializa o scraper
        scraper = WebScraper()

        # Começa no site do Tabnet

        # Clica em Epidemiológicas e Morbidade
        scraper.clicar_elemento('//*[@id="elementor-tab-title-2153"]')
        scraper.aguardar_visibilidade('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')

        # Clica Sistema de Informação do Câncer - SISCAN (colo do útero e mama)
        scraper.clicar_elemento('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')
        scraper.aguardar_visibilidade('//*[@id="infesq"]/input[8]')

        # Clica em Mamografia - Por local de atendimento
        scraper.clicar_elemento('//*[@id="infesq"]/input[8]')
        scraper.aguardar_visibilidade('//*[@id="mySelect"]/option[23]')

        # Clica em Rio Grande do Sul
        scraper.clicar_elemento('//*[@id="mySelect"]/option[23]')
        scraper.aguardar_visibilidade('//*[@id="L"]/option[7]')

        # Escolhe a opção de linha: Faixa etária
        scraper.clicar_elemento('//*[@id="L"]/option[7]')

        # Escolhe a opção de coluna: Ano de Competência
        scraper.clicar_elemento('//*[@id="C"]/option[2]')

        # Escolhe os anos: 2023, 2022, 2021, 2020, 2019
        scraper.clicar_elemento('//*[@id="A"]/option[1]')
        scraper.selecionar_multiplas_opcoes('//*[@id="A"]/option[2]', '//*[@id="A"]/option[3]', '//*[@id="A"]/option[4]', '//*[@id="A"]/option[5]', '//*[@id="A"]/option[6]')
        scraper.aguardar_visibilidade('//*[@id="fig2"]')

        # ------------ MUNICÍPIOS ------------
        # Cruz Alta: //*[@id="S2"]/option[127]
        # Giruá:     //*[@id="S2"]/option[183]
        # Ijuí:      //*[@id="S2"]/option[204]
        # Panambi:   //*[@id="S2"]/option[299]

        # Clica no filtro de município e escreve "Cruz Alta"
        scraper.clicar_elemento('//*[@id="fig2"]')
        scraper.clicar_elemento('//*[@id="S2"]/option[1]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Cruz Alta")
        scraper.aguardar_visibilidade('//*[@id="S2"]/option[127]')
        scraper.clicar_elemento('//*[@id="S2"]/option[127]')

        # Escolhe a opção: feminino
        scraper.clicar_elemento('//*[@id="fig8"]')
        scraper.clicar_elemento('//*[@id="S8"]/option[1]')
        scraper.clicar_elemento('//*[@id="S8"]/option[3]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Giruá"
        scraper.clicar_elemento('//*[@id="S2"]/option[127]')
        scraper.limpar_campo('//*[@id="pesqmes2"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Giruá")
        scraper.clicar_elemento('//*[@id="S2"]/option[183]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Ijuí"
        scraper.clicar_elemento('//*[@id="S2"]/option[183]')
        scraper.limpar_campo('//*[@id="pesqmes2"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Ijuí")
        scraper.clicar_elemento('//*[@id="S2"]/option[204]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Panambi"
        scraper.clicar_elemento('//*[@id="S2"]/option[204]')
        scraper.limpar_campo('//*[@id="pesqmes2"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Panambi")
        scraper.clicar_elemento('//*[@id="S2"]/option[299]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Trocar para a nova janela/aba (Cruz Alta)
        scraper.trocar_para_nova_janela(3)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.clicar_elemento('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        scraper.tirar_screenshot('/content/tabela_cruz_alta_atendimento.png')

        # Trocar para a nova janela/aba (Giruá)
        scraper.trocar_para_nova_janela(2)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.clicar_elemento('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        scraper.tirar_screenshot('/content/tabela_giruá_atendimento.png')

        # Trocar para a nova janela/aba (Ijuí)
        scraper.trocar_para_nova_janela(1)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.clicar_elemento('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        scraper.tirar_screenshot('/content/tabela_ijuí_atendimento.png')

        # Trocar para a nova janela/aba (Panambi)
        scraper.trocar_para_nova_janela(4)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.clicar_elemento('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        scraper.tirar_screenshot('/content/tabela_panambi_atendimento.png')

        # Aguarda alguns segundos antes de fechar o navegador para garantir que os arquivos serão baixados
        scraper.aguardar(2)

        scraper.fechar_navegador()

    except Exception as e:
        print(f"Erro no processo principal: {e}")


def residencia():
    try:
        # Inicializa o scraper
        scraper = WebScraper()

        # Começa no site do Tabnet

        # Clica em Epidemiológicas e Morbidade
        scraper.clicar_elemento('//*[@id="elementor-tab-title-2153"]')
        scraper.aguardar_visibilidade('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')

        # Clica Sistema de Informação do Câncer - SISCAN (colo do útero e mama)
        scraper.clicar_elemento('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')
        scraper.aguardar_visibilidade('//*[@id="infesq"]/input[7]')

        # Clica em Mamografia - Por local de residência
        scraper.clicar_elemento('//*[@id="infesq"]/input[7]')
        scraper.aguardar_visibilidade('//*[@id="mySelect"]/option[23]')

        # Clica em Rio Grande do Sul
        scraper.clicar_elemento('//*[@id="mySelect"]/option[23]')
        scraper.aguardar_visibilidade('//*[@id="L"]/option[4]')

        # Escolhe a opção de linha: Faixa etária
        scraper.clicar_elemento('//*[@id="L"]/option[4]')

        # Escolhe a opção de coluna: Ano de Competência
        scraper.clicar_elemento('//*[@id="C"]/option[2]')

        # Escolhe os anos: 2023, 2022, 2021, 2020, 2019
        scraper.clicar_elemento('//*[@id="A"]/option[1]')
        scraper.selecionar_multiplas_opcoes('//*[@id="A"]/option[2]', '//*[@id="A"]/option[3]', '//*[@id="A"]/option[4]', '//*[@id="A"]/option[5]', '//*[@id="A"]/option[6]')
        scraper.aguardar_visibilidade('//*[@id="fig1"]')

        # ------------ MUNICÍPIOS ------------
        # Cruz Alta: //*[@id="S1"]/option[127]
        # Giruá:     //*[@id="S1"]/option[183]
        # Ijuí:      //*[@id="S1"]/option[204]
        # Panambi:   //*[@id="S1"]/option[299]

        # Clica no filtro de município e escreve "Cruz Alta"
        scraper.clicar_elemento('//*[@id="fig1"]')
        scraper.clicar_elemento('//*[@id="S1"]/option[1]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes1"]', "Cruz Alta")
        scraper.aguardar_visibilidade('//*[@id="S1"]/option[127]')
        scraper.clicar_elemento('//*[@id="S1"]/option[127]')

        # Escolhe a opção: feminino
        scraper.clicar_elemento('//*[@id="fig6"]')
        scraper.clicar_elemento('//*[@id="S6"]/option[1]')
        scraper.clicar_elemento('//*[@id="S6"]/option[3]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Giruá"
        scraper.clicar_elemento('//*[@id="S1"]/option[127]')
        scraper.limpar_campo('//*[@id="pesqmes1"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes1"]', "Giruá")
        scraper.clicar_elemento('//*[@id="S1"]/option[183]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Ijuí"
        scraper.clicar_elemento('//*[@id="S1"]/option[183]')
        scraper.limpar_campo('//*[@id="pesqmes1"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes1"]', "Ijuí")
        scraper.clicar_elemento('//*[@id="S1"]/option[204]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Panambi"
        scraper.clicar_elemento('//*[@id="S1"]/option[204]')
        scraper.limpar_campo('//*[@id="pesqmes1"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes1"]', "Panambi")
        scraper.clicar_elemento('//*[@id="S1"]/option[299]')

        # Clicar no botão de gerar tabela (mostra)
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Trocar para a nova janela/aba (Cruz Alta)
        scraper.trocar_para_nova_janela(3)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.clicar_elemento('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        scraper.tirar_screenshot('/content/tabela_cruz_alta_residencia.png')

        # Trocar para a nova janela/aba (Giruá)
        scraper.trocar_para_nova_janela(2)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.clicar_elemento('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        scraper.tirar_screenshot('/content/tabela_giruá_residencia.png')

        # Trocar para a nova janela/aba (Ijuí)
        scraper.trocar_para_nova_janela(1)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.clicar_elemento('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        scraper.tirar_screenshot('/content/tabela_ijuí_residencia.png')

        # Trocar para a nova janela/aba (Panambi)
        scraper.trocar_para_nova_janela(4)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.clicar_elemento('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        scraper.tirar_screenshot('/content/tabela_panambi_residencia.png')

        # Aguarda alguns segundos antes de fechar o navegador para garantir que os arquivos serão baixados
        scraper.aguardar(2)

        scraper.fechar_navegador()

    except Exception as e:
        print(f"Erro no processo principal: {e}")


def limpar_diretorio():
    # Itera sobre todos os arquivos no diretório '/content'
    for arquivo in os.listdir('/content'):
        # Monta o caminho completo para o arquivo
        caminho_arquivo = os.path.join('/content', arquivo)
        try:
            # Verifica se o caminho corresponde a um arquivo e não a um diretório
            if os.path.isfile(caminho_arquivo):
                # Exclui o arquivo
                os.unlink(caminho_arquivo)
        except Exception as e:
            # Caso ocorra algum erro ao tentar excluir o arquivo, imprime o erro
            print(f"Erro ao excluir {caminho_arquivo}: {e}")


def nomear_arquivos():
    # Diretório onde os arquivos foram baixados
    diretorio = '/content'

    # Lista na ordem desejada
    ordem_desejada = [
        "cruzAltaAtendimento",
        "giruaAtendimento",
        "ijuiAtendimento",
        "panambiAtendimento",
        "cruzAltaResidencia",
        "giruaResidencia",
        "ijuiResidencia",
        "panambiResidencia"
    ]

    # Lista os arquivos baixados no diretório
    arquivos_baixados = os.listdir(diretorio)

    # Filtra apenas os arquivos CSV baixados pelas funções atendimento e residencia
    arquivos_csv = [arquivo for arquivo in arquivos_baixados if arquivo.endswith('.csv')]

    # Ordena os arquivos pela data de modificação (do mais antigo ao mais recente)
    arquivos_csv.sort(key=lambda x: os.path.getmtime(os.path.join(diretorio, x)))

    # Verifica se o número de arquivos CSV corresponde ao número de nomes desejados
    if len(arquivos_csv) != len(ordem_desejada):
        raise ValueError("O número de arquivos CSV não corresponde ao número de nomes desejados")

    # Carrega os arquivos CSV na ordem desejada e atribui a variáveis individuais
    for nome_variavel, arquivo_csv in zip(ordem_desejada, arquivos_csv):
        caminho_arquivo = os.path.join(diretorio, arquivo_csv)
        # Ajuste o sep e o encoding conforme necessário
        globals()[nome_variavel] = pd.read_csv(caminho_arquivo, sep=';', encoding='unicode_escape')


def concatenar():
    # Adicionando uma nova coluna de índice chamada 'Município' nas tabelas: Exames por Ano segundo Faixa Etária - Local de Atendimento
    cruzAltaAtendimento['Município'] = 'Cruz Alta'
    giruaAtendimento['Município'] = 'Giruá'
    ijuiAtendimento['Município'] = 'Ijuí'
    panambiAtendimento['Município'] = 'Panambi'

    # Adicionando uma nova coluna de índice chamada 'Município' nas tabelas: Exames por Ano segundo Faixa Etária - Local de Residência
    cruzAltaResidencia['Município'] = 'Cruz Alta'
    giruaResidencia['Município'] = 'Giruá'
    ijuiResidencia['Município'] = 'Ijuí'
    panambiResidencia['Município'] = 'Panambi'

    # Concatena tudo sem repetir o nome da coluna (Local de Atendimento)
    concatenadaAtendimento = pd.concat([cruzAltaAtendimento, giruaAtendimento, ijuiAtendimento, panambiAtendimento], ignore_index=False)
    concatenadaAtendimento.set_index(['Município', 'Faixa etária'], inplace=True)
    # Baixa o arquivo em formato de excel
    concatenadaAtendimento.to_excel('Exames por Ano segundo Faixa Etária - Local de Atendimento (Somente mulheres).xlsx')

    # Concatena tudo sem repetir o nome da coluna (Local de Residência)
    concatenadaResidencia = pd.concat([cruzAltaResidencia, giruaResidencia, ijuiResidencia, panambiResidencia], ignore_index=False)
    concatenadaResidencia.set_index(['Município', 'Faixa etária'], inplace=True)
    # Baixa o arquivo em formato de excel
    concatenadaResidencia.to_excel('Exames por Ano segundo Faixa Etária - Local de Residência (Somente mulheres).xlsx')

    print('\nTabelas concatenadas disponíveis no diretório:\n')
    print('Exames por Ano segundo Faixa Etária - Local de Atendimento (Somente mulheres)\n')
    print('Exames por Ano segundo Faixa Etária - Local de Residência (Somente mulheres)')


# Main
if __name__ == "__main__":
    # Começa o código limpando o diretório para que não ocorra confusão em caso de múltiplos testes
    limpar_diretorio()
    # Exames por Ano segundo Faixa Etária - Local de Atendimento (Somente mulheres)
    atendimento()
    # Exames por Ano segundo Faixa Etária - Local de Residência (Somente mulheres)
    residencia()
    # Nomea os arquivos por município e local (atendimento ou residência)
    nomear_arquivos()
    # Concatena os quatro municípios em uma só tabela (atendimento e residência)
    concatenar()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Tabelas concatenadas disponíveis no diretório:

Exames por Ano segundo Faixa Etária - Local de Atendimento (Somente mulheres)

Exames por Ano segundo Faixa Etária - Local de Residência (Somente mulheres)
